## 서울시 자치구별 5대범죄 검거율 지도 시각화

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import json
import folium
from folium.features import DivIcon

import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname="C:/Windows/Fonts/malgun.ttf").get_name()
plt.rc('font', family=font_name)

mpl.rcParams["axes.unicode_minus"] = False

In [2]:
# 전체 코드 출력
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [3]:
# 한글 문제
# matplotlit의 기본 폰트에서 한글 지원되지 않기 때문에
# matplotlib의 폰트 변경 필요
import platform

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':  # 맥OS 
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':  # 윈도우
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system...  sorry~~~')

### 2017,2018,2019 범죄 검거율 데이터 전처리

In [4]:
crime_rates=pd.read_csv('../[0]data/서울시_5대범죄_발생_검거.csv',index_col=0)
crime_rates

,지역구,발생합계,살인발생,강도발생,강간강제추행발생,절도발생,폭력발생,검거합계,살인검거,강도검거,강간강제추행검거,절도검거,폭력검거
기간,,,,,,,,,,,,,
2017,강남구,7720,7,20,607,3130,3956,5321,7,24,496,1383,3411
2018,강남구,7513,10,14,600,3004,3885,5196,13,10,523,1317,3333
2019,강남구,7304,5,5,666,2970,3658,5069,3,6,562,1339,3159
2017,종로구,4057,7,10,253,1644,2143,4437,9,15,1412,905,2096
2017,관악구,5525,8,7,379,1979,3152,4154,7,6,323,1152,2666
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019,성동구,2512,6,9,133,970,1394,1838,5,10,96,511,1216
2018,성동구,2457,2,5,126,999,1325,1787,2,8,102,487,1188
2018,도봉구,2249,1,5,113,781,1349,1676,1,3,117,431,1124


In [5]:
crime_rates['살인검거']=pd.to_numeric(crime_rates['살인검거'], errors='coerce')

In [6]:
crime_rates.head(40)
crime_rates.tail(40)

,지역구,발생합계,살인발생,강도발생,강간강제추행발생,절도발생,폭력발생,검거합계,살인검거,강도검거,강간강제추행검거,절도검거,폭력검거
기간,,,,,,,,,,,,,
2017,강남구,7720,7,20,607,3130,3956,5321,7.0,24,496,1383,3411
2018,강남구,7513,10,14,600,3004,3885,5196,13.0,10,523,1317,3333
2019,강남구,7304,5,5,666,2970,3658,5069,3.0,6,562,1339,3159
2017,종로구,4057,7,10,253,1644,2143,4437,9.0,15,1412,905,2096
2017,관악구,5525,8,7,379,1979,3152,4154,7.0,6,323,1152,2666
2019,종로구,3846,4,7,238,1515,2082,4117,5.0,8,1143,1069,1892
2018,송파구,5807,5,6,309,2352,3135,4051,6.0,8,254,1034,2749
2017,영등포구,5969,15,5,455,2304,3190,4012,13.0,5,267,1108,2619
2017,강서구,5135,11,4,316,1888,2916,3927,11.0,6,253,1113,2544


,지역구,발생합계,살인발생,강도발생,강간강제추행발생,절도발생,폭력발생,검거합계,살인검거,강도검거,강간강제추행검거,절도검거,폭력검거
기간,,,,,,,,,,,,,
2019,강북구,3838,6,3,176,1229,2424,3023,6.0,2,145,664,2206
2019,은평구,3880,3,1,194,1565,2117,3000,3.0,1,162,977,1857
2018,동대문구,3680,2,5,145,1447,2081,2968,2.0,4,112,930,1920
2019,노원구,4153,4,3,184,1498,2464,2964,4.0,3,158,681,2118
2019,강동구,4014,6,15,180,1650,2163,2955,5.0,14,155,907,1874
2017,용산구,4060,1,5,327,1346,2381,2936,1.0,4,289,626,2016
2018,은평구,3590,8,1,188,1278,2115,2863,8.0,1,169,816,1869
2017,중구,4184,1,9,267,1846,2061,2851,NaN,9,149,889,1804
2017,강북구,3393,8,7,208,1014,2156,2824,8.0,5,153,635,2023


In [7]:
crime_rates=crime_rates.fillna(0)

In [8]:
crime_rates.head(40)
crime_rates.tail(40)

,지역구,발생합계,살인발생,강도발생,강간강제추행발생,절도발생,폭력발생,검거합계,살인검거,강도검거,강간강제추행검거,절도검거,폭력검거
기간,,,,,,,,,,,,,
2017,강남구,7720,7,20,607,3130,3956,5321,7.0,24,496,1383,3411
2018,강남구,7513,10,14,600,3004,3885,5196,13.0,10,523,1317,3333
2019,강남구,7304,5,5,666,2970,3658,5069,3.0,6,562,1339,3159
2017,종로구,4057,7,10,253,1644,2143,4437,9.0,15,1412,905,2096
2017,관악구,5525,8,7,379,1979,3152,4154,7.0,6,323,1152,2666
2019,종로구,3846,4,7,238,1515,2082,4117,5.0,8,1143,1069,1892
2018,송파구,5807,5,6,309,2352,3135,4051,6.0,8,254,1034,2749
2017,영등포구,5969,15,5,455,2304,3190,4012,13.0,5,267,1108,2619
2017,강서구,5135,11,4,316,1888,2916,3927,11.0,6,253,1113,2544


,지역구,발생합계,살인발생,강도발생,강간강제추행발생,절도발생,폭력발생,검거합계,살인검거,강도검거,강간강제추행검거,절도검거,폭력검거
기간,,,,,,,,,,,,,
2019,강북구,3838,6,3,176,1229,2424,3023,6.0,2,145,664,2206
2019,은평구,3880,3,1,194,1565,2117,3000,3.0,1,162,977,1857
2018,동대문구,3680,2,5,145,1447,2081,2968,2.0,4,112,930,1920
2019,노원구,4153,4,3,184,1498,2464,2964,4.0,3,158,681,2118
2019,강동구,4014,6,15,180,1650,2163,2955,5.0,14,155,907,1874
2017,용산구,4060,1,5,327,1346,2381,2936,1.0,4,289,626,2016
2018,은평구,3590,8,1,188,1278,2115,2863,8.0,1,169,816,1869
2017,중구,4184,1,9,267,1846,2061,2851,0.0,9,149,889,1804
2017,강북구,3393,8,7,208,1014,2156,2824,8.0,5,153,635,2023


In [9]:
crime_rates=crime_rates.reset_index()

In [10]:
crime_rates=crime_rates.set_index(['기간','지역구'])

In [11]:
crime_rates

발생합계  살인발생  강도발생  강간강제추행발생  절도발생  폭력발생  검거합계  살인검거  강도검거  강간강제추행검거  \
기간   지역구                                                                       
2017 강남구  7720     7    20       607  3130  3956  5321   7.0    24       496   
2018 강남구  7513    10    14       600  3004  3885  5196  13.0    10       523   
2019 강남구  7304     5     5       666  2970  3658  5069   3.0     6       562   
2017 종로구  4057     7    10       253  1644  2143  4437   9.0    15      1412   
     관악구  5525     8     7       379  1979  3152  4154   7.0     6       323   
...        ...   ...   ...       ...   ...   ...   ...   ...   ...       ...   
2019 성동구  2512     6     9       133   970  1394  1838   5.0    10        96   
2018 성동구  2457     2     5       126   999  1325  1787   2.0     8       102   
     도봉구  2249     1     5       113   781  1349  1676   1.0     3       117   
2017 도봉구  1999     1     3       104   633  1258  1652   1.0     3       115   
2019 도봉구  2110     1     5        90   790  1224  1497   1.0     5        83   

          절도검거  폭력검거  
기간   지역구              
2017 강남구  1383  3411  
2018 강남구  1317  3333  
2019 강남구  1339  3159  
2017 종로구   905  2096  
     관악구  1152  2666  
...        ...   ...  
2019 성동구   511  1216  
2018 성동구   487  1188  
     도봉구   431  1124  
2017 도봉구   417  1116  
2019 도봉구   394  1014  

[75 rows x 12 columns]

In [12]:
crime_rates_total=crime_rates[['발생합계','검거합계']]

In [13]:
crime_rates_total

발생합계  검거합계
기간   지역구            
2017 강남구  7720  5321
2018 강남구  7513  5196
2019 강남구  7304  5069
2017 종로구  4057  4437
     관악구  5525  4154
...        ...   ...
2019 성동구  2512  1838
2018 성동구  2457  1787
     도봉구  2249  1676
2017 도봉구  1999  1652
2019 도봉구  2110  1497

[75 rows x 2 columns]

In [14]:
crime_rates_total=crime_rates_total.sort_index()
crime_rates_total['검거율'] =  ((crime_rates_total['발생합계'] / crime_rates_total['검거합계'])*100).round()
crime_rates_total

발생합계  검거합계    검거율
기간   지역구                   
2017 강남구  7720  5321  145.0
     강동구  4261  3245  131.0
     강북구  3393  2824  120.0
     강서구  5135  3927  131.0
     관악구  5525  4154  133.0
...        ...   ...    ...
2019 용산구  3313  2611  127.0
     은평구  3880  3000  129.0
     종로구  3846  4117   93.0
     중구   4327  2804  154.0
     중랑구  4268  3138  136.0

[75 rows x 3 columns]

In [15]:
crime_rates2017 = crime_rates_total.loc[2017]
crime_rates2018 = crime_rates_total.loc[2018]
crime_rates2019 = crime_rates_total.loc[2019]

In [16]:
crime_rates2017
crime_rates2018
crime_rates2019

,발생합계,검거합계,검거율
지역구,,,
강남구,7720,5321,145.0
강동구,4261,3245,131.0
강북구,3393,2824,120.0
강서구,5135,3927,131.0
관악구,5525,4154,133.0
광진구,4646,3289,141.0
구로구,4895,3408,144.0
금천구,3265,2572,127.0
노원구,4209,3047,138.0


,발생합계,검거합계,검거율
지역구,,,
강남구,7513,5196,145.0
강동구,3919,2789,141.0
강북구,3437,2743,125.0
강서구,4629,3469,133.0
관악구,5026,3653,138.0
광진구,3915,2789,140.0
구로구,4810,3338,144.0
금천구,3293,2567,128.0
노원구,4007,2725,147.0


,발생합계,검거합계,검거율
지역구,,,
강남구,7304,5069,144.0
강동구,4014,2955,136.0
강북구,3838,3023,127.0
강서구,4924,3543,139.0
관악구,5328,3810,140.0
광진구,4011,2816,142.0
구로구,4707,3291,143.0
금천구,3105,2343,133.0
노원구,4153,2964,140.0


In [17]:
crime_rates1=pd.merge(crime_rates2017,crime_rates2018,on='지역구')

In [18]:
crime_rates=pd.merge(crime_rates1,crime_rates2019,on='지역구')

In [19]:
crime_rates.rename(columns = {'발생합계_x' : '발생합계(2017)','검거합계_x':'검거합계(2017)','검거율_x':'검거율(2017)',
                             '발생합계_y' : '발생합계(2018)','검거합계_y':'검거합계(2018)','검거율_y':'검거율(2018)',
                             '발생합계' : '발생합계(2019)','검거합계':'검거합계(2019)','검거율':'검거율(2019)'},inplace = True)

In [20]:
crime_rates

,발생합계(2017),검거합계(2017),검거율(2017),발생합계(2018),검거합계(2018),검거율(2018),발생합계(2019),검거합계(2019),검거율(2019)
지역구,,,,,,,,,
강남구,7720,5321,145.0,7513,5196,145.0,7304,5069,144.0
강동구,4261,3245,131.0,3919,2789,141.0,4014,2955,136.0
강북구,3393,2824,120.0,3437,2743,125.0,3838,3023,127.0
강서구,5135,3927,131.0,4629,3469,133.0,4924,3543,139.0
관악구,5525,4154,133.0,5026,3653,138.0,5328,3810,140.0
광진구,4646,3289,141.0,3915,2789,140.0,4011,2816,142.0
구로구,4895,3408,144.0,4810,3338,144.0,4707,3291,143.0
금천구,3265,2572,127.0,3293,2567,128.0,3105,2343,133.0
노원구,4209,3047,138.0,4007,2725,147.0,4153,2964,140.0


In [21]:
crime_rates['검거율평균']=((crime_rates['검거율(2017)']+crime_rates['검거율(2018)']+crime_rates['검거율(2019)'])/3).round()

In [22]:
crime_rates.sort_values("검거율평균",ascending=False)

,발생합계(2017),검거합계(2017),검거율(2017),발생합계(2018),검거합계(2018),검거율(2018),발생합계(2019),검거합계(2019),검거율(2019),검거율평균
지역구,,,,,,,,,,
동작구,3330,2242,149.0,3100,2090,148.0,3400,2195,155.0,151.0
영등포구,5969,4012,149.0,5840,3922,149.0,5820,3787,154.0,151.0
마포구,5278,3499,151.0,5172,3569,145.0,4842,3170,153.0,150.0
중구,4184,2851,147.0,4030,2679,150.0,4327,2804,154.0,150.0
양천구,3882,2660,146.0,3515,2459,143.0,3214,2124,151.0,147.0
서초구,4708,3237,145.0,4726,3183,148.0,5542,3750,148.0,147.0
송파구,5576,3870,144.0,5807,4051,143.0,5698,3799,150.0,146.0
강남구,7720,5321,145.0,7513,5196,145.0,7304,5069,144.0,145.0
구로구,4895,3408,144.0,4810,3338,144.0,4707,3291,143.0,144.0


In [23]:
crime_rates=crime_rates.reset_index()

In [24]:
geo_path = ('../[0]data/skorea_municipalities_geo_simple.json')

geo_str = json.load(open(geo_path, encoding='utf-8'))
geo_str

FileNotFoundError: [Errno 2] No such file or directory: '../[0]data/skorea_municipalities_geo_simple.json'

In [ ]:
crime_arrest_map = folium.Map(location = [37.5502, 126.982],zoom_start = 10.5)

In [ ]:
crime_arrest_map

In [ ]:
folium.Choropleth(
    geo_data = geo_str,
    data = crime_rates,
    columns = ['지역구', '검거율평균'],
    fill_color='YlGn',
    key_on='properties.name',
    fill_opacity=0.8,
    ).add_to(crime_arrest_map)

In [ ]:
crime_arrest_map

In [ ]:
text = '동작구'
circle_lat = 37.5092
circle_lon = 126.967


folium.map.Marker(
    [circle_lat, circle_lon],
    icon=DivIcon(
        icon_size=(10,6),
        icon_anchor=(0,0), 
        html='<div style="font-size: pt; font-weight:bold;">%s</div>' % text,
        )
    ).add_to(crime_arrest_map)

In [ ]:
text = '중구'
circle_lat = 37.5650
circle_lon = 126.999


folium.map.Marker(
    [circle_lat, circle_lon],
    icon=DivIcon(
        icon_size=(10,6),
        icon_anchor=(0,0), 
        html='<div style="font-size: pt; font-weight:bold;">%s</div>' % text,
        )
    ).add_to(crime_arrest_map)

In [ ]:
text = '용산구'
circle_lat = 37.5450
circle_lon = 126.980


folium.map.Marker(
    [circle_lat, circle_lon],
    icon=DivIcon(
        icon_size=(10,6),
        icon_anchor=(0,0), 
        html='<div style="font-size: pt; font-weight:bold;">%s</div>' % text,
        )
    ).add_to(crime_arrest_map)

In [ ]:
text = '성동구'
circle_lat = 37.5600
circle_lon = 127.040


folium.map.Marker(
    [circle_lat, circle_lon],
    icon=DivIcon(
        icon_size=(10,6),
        icon_anchor=(0,0), 
        html='<div style="font-size: pt; font-weight:bold;">%s</div>' % text,
        )
    ).add_to(crime_arrest_map)

In [ ]:
text = '서초구'
circle_lat = 37.5000
circle_lon = 127.000


folium.map.Marker(
    [circle_lat, circle_lon],
    icon=DivIcon(
        icon_size=(10,6),
        icon_anchor=(0,0), 
        html='<div style="font-size: pt; font-weight:bold;">%s</div>' % text,
        )
    ).add_to(crime_arrest_map)

In [ ]:
text = '강남구'
circle_lat = 37.5100
circle_lon = 127.050


folium.map.Marker(
    [circle_lat, circle_lon],
    icon=DivIcon(
        icon_size=(10,6),
        icon_anchor=(0,0), 
        html='<div style="font-size: pt; font-weight:bold;">%s</div>' % text,
        )
    ).add_to(crime_arrest_map)

In [ ]:
text = '송파구'
circle_lat = 37.5100
circle_lon = 127.120


folium.map.Marker(
    [circle_lat, circle_lon],
    icon=DivIcon(
        icon_size=(10,6),
        icon_anchor=(0,0), 
        html='<div style="font-size: pt; font-weight:bold;">%s</div>' % text,
        )
    ).add_to(crime_arrest_map)

In [ ]:
text = '강동구'
circle_lat = 37.5600
circle_lon = 127.140


folium.map.Marker(
    [circle_lat, circle_lon],
    icon=DivIcon(
        icon_size=(10,6),
        icon_anchor=(0,0), 
        html='<div style="font-size: pt; font-weight:bold;">%s</div>' % text,
        )
    ).add_to(crime_arrest_map)

In [ ]:
text = '광진구'
circle_lat = 37.5600
circle_lon = 127.090


folium.map.Marker(
    [circle_lat, circle_lon],
    icon=DivIcon(
        icon_size=(10,6),
        icon_anchor=(0,0), 
        html='<div style="font-size: pt; font-weight:bold;">%s</div>' % text,
        )
    ).add_to(crime_arrest_map)

In [ ]:
text = '중랑구'
circle_lat = 37.6050
circle_lon = 127.090


folium.map.Marker(
    [circle_lat, circle_lon],
    icon=DivIcon(
        icon_size=(10,6),
        icon_anchor=(0,0), 
        html='<div style="font-size: pt; font-weight:bold;">%s</div>' % text,
        )
    ).add_to(crime_arrest_map)

In [ ]:
text = '노원구'
circle_lat = 37.6650
circle_lon = 127.075


folium.map.Marker(
    [circle_lat, circle_lon],
    icon=DivIcon(
        icon_size=(10,6),
        icon_anchor=(0,0), 
        html='<div style="font-size: pt; font-weight:bold;">%s</div>' % text,
        )
    ).add_to(crime_arrest_map)

In [ ]:
text = '도봉구'
circle_lat = 37.6750
circle_lon = 127.033


folium.map.Marker(
    [circle_lat, circle_lon],
    icon=DivIcon(
        icon_size=(10,6),
        icon_anchor=(0,0), 
        html='<div style="font-size: pt; font-weight:bold;">%s</div>' % text,
        )
    ).add_to(crime_arrest_map)

In [ ]:
text = '강북구'
circle_lat = 37.6650
circle_lon = 127.000


folium.map.Marker(
    [circle_lat, circle_lon],
    icon=DivIcon(
        icon_size=(10,6),
        icon_anchor=(0,0), 
        html='<div style="font-size: pt; font-weight:bold;">%s</div>' % text,
        )
    ).add_to(crime_arrest_map)

In [ ]:
text = '성북구'
circle_lat = 37.6100
circle_lon = 127.020


folium.map.Marker(
    [circle_lat, circle_lon],
    icon=DivIcon(
        icon_size=(10,6),
        icon_anchor=(0,0), 
        html='<div style="font-size: pt; font-weight:bold;">%s</div>' % text,
        )
    ).add_to(crime_arrest_map)

In [ ]:
text = '동대문구'
circle_lat = 37.5990
circle_lon = 127.060


folium.map.Marker(
    [circle_lat, circle_lon],
    icon=DivIcon(
        icon_size=(10,6),
        icon_anchor=(0,0), 
        html='<div style="font-size: pt; font-weight:bold;">%s</div>' % text,
        )
    ).add_to(crime_arrest_map)

In [ ]:
text = '종로구'
circle_lat = 37.6100
circle_lon = 126.970


folium.map.Marker(
    [circle_lat, circle_lon],
    icon=DivIcon(
        icon_size=(10,6),
        icon_anchor=(0,0), 
        html='<div style="font-size: pt; font-weight:bold;">%s</div>' % text,
        )
    ).add_to(crime_arrest_map)

In [ ]:
text = '은평구'
circle_lat = 37.6300
circle_lon = 126.930

folium.map.Marker(
    [circle_lat, circle_lon],
    icon=DivIcon(
        icon_size=(10,6),
        icon_anchor=(0,0), 
        html='<div style="font-size: pt; font-weight:bold;">%s</div>' % text,
        )
    ).add_to(crime_arrest_map)

In [ ]:
text = '서대문구'
circle_lat = 37.5930
circle_lon = 126.940


folium.map.Marker(
    [circle_lat, circle_lon],
    icon=DivIcon(
        icon_size=(10,6),
        icon_anchor=(0,0), 
        html='<div style="font-size: pt; font-weight:bold;">%s</div>' % text,
        )
    ).add_to(crime_arrest_map)

In [ ]:
text = '마포구'
circle_lat = 37.5680
circle_lon = 126.910


folium.map.Marker(
    [circle_lat, circle_lon],
    icon=DivIcon(
        icon_size=(10,6),
        icon_anchor=(0,0), 
        html='<div style="font-size: pt; font-weight:bold;">%s</div>' % text,
        )
    ).add_to(crime_arrest_map)

In [ ]:
text = '강서구'
circle_lat = 37.5700
circle_lon = 126.830


folium.map.Marker(
    [circle_lat, circle_lon],
    icon=DivIcon(
        icon_size=(10,6),
        icon_anchor=(0,0), 
        html='<div style="font-size: pt; font-weight:bold;">%s</div>' % text,
        )
    ).add_to(crime_arrest_map)

In [ ]:
text = '양천구'
circle_lat = 37.5430
circle_lon = 126.872


folium.map.Marker(
    [circle_lat, circle_lon],
    icon=DivIcon(
        icon_size=(10,6),
        icon_anchor=(0,0), 
        html='<div style="font-size: pt; font-weight:bold;">%s</div>' % text,
        )
    ).add_to(crime_arrest_map)

In [ ]:
text = '영등포구'
circle_lat = 37.5350
circle_lon = 126.905


folium.map.Marker(
    [circle_lat, circle_lon],
    icon=DivIcon(
        icon_size=(10,6),
        icon_anchor=(0,0), 
        html='<div style="font-size: pt; font-weight:bold;">%s</div>' % text,
        )
    ).add_to(crime_arrest_map)

In [ ]:
text = '구로구'
circle_lat = 37.5080
circle_lon = 126.858


folium.map.Marker(
    [circle_lat, circle_lon],
    icon=DivIcon(
        icon_size=(10,6),
        icon_anchor=(0,0), 
        html='<div style="font-size: pt; font-weight:bold;">%s</div>' % text,
        )
    ).add_to(crime_arrest_map)

In [ ]:
text = '금천구'
circle_lat = 37.4750
circle_lon = 126.895


folium.map.Marker(
    [circle_lat, circle_lon],
    icon=DivIcon(
        icon_size=(10,6),
        icon_anchor=(0,0), 
        html='<div style="font-size: pt; font-weight:bold;">%s</div>' % text,
        )
    ).add_to(crime_arrest_map)

In [ ]:
text = '관악구'
circle_lat = 37.4800
circle_lon = 126.940


folium.map.Marker(
    [circle_lat, circle_lon],
    icon=DivIcon(
        icon_size=(10,6),
        icon_anchor=(0,0), 
        html='<div style="font-size: pt; font-weight:bold;">%s</div>' % text,
        )
    ).add_to(crime_arrest_map)

In [ ]:
crime_arrest_map